In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

Это товар не из топ-5000. Мы его использовали для того, чтобы не терять пользователей, которые покупали товары не из топ-5000. 
Используя этот товар мы смещаем рекомендации в сторону этого товара. Этот товар удалить можно, но не очень целесообразно, поскольку нарушится история покупок пользователей и мы забудем пользователей.

In [16]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [69]:
def precision_at_k(boughted, recommended, k=5):
    recommended = recommended[:k]

    precision = np.sum(np.isin(boughted, recommended)) / len(recommended)
    
    return precision

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [13]:
items = data_train.item_id.unique()
sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
weights = sales['sales_value'].apply(lambda x: np.log(x)).values

In [14]:
weights = np.array([x if x >= 0 else 0 for x in weights])

In [20]:
total = weights.sum()
weights = weights / total

In [21]:
def weighted_random_recommendation(items, weights, n=5):
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False, p=weights)
    
    return recs.tolist()

Сделайте предсказания

In [22]:
%%time

result['weighted_random'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))
result.head()

Wall time: 7.54 s


,user_id,actual,weighted_random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6389889, 1130969, 986754, 857332, 1052807]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12384141, 902811, 9529450, 5579981, 13116018]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1111528, 837745, 936491, 976471, 1095735]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[8068430, 8020245, 1087999, 281655, 909102]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13006928, 1049048, 831648, 13213123, 600264]"


Какие веса можно ещё использовать:
   - логарифм объёмов продаж в натуральном выражении
   - можно добавить группировку по времени(т.е. создать новый категориальный признак time+metrics, где
     time - это временной отрезок, metrics-метрика, пропорционально которой рекомендуем) и выстроить рекомендации пропорционально тем временным окнам, которые создали. 
   - можно рекомендовать недавно просмотренные(купленные товары). Т.е. тут будет вес пропорционален "глубине" истории продаж.

Посмотрим на популярное.

In [24]:
def popularity_recommendation(data, n=5):
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [25]:
%%time

#рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head()

Wall time: 381 ms


,user_id,actual,weighted_random,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[6389889, 1130969, 986754, 857332, 1052807]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12384141, 902811, 9529450, 5579981, 13116018]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1111528, 837745, 936491, 976471, 1095735]","[6534178, 6533889, 1029743, 6534166, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[8068430, 8020245, 1087999, 281655, 909102]","[6534178, 6533889, 1029743, 6534166, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[13006928, 1049048, 831648, 13213123, 600264]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [27]:
type(result['weighted_random'][0])

list

In [29]:
result.apply(lambda x: precision_at_k(x['weighted_random'], x['actual'],  5), axis=1).mean()

0.0

In [30]:
result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()

0.027138426379366695

По оффлайновой метрике (ml-метрике) выше результат у популярного. Однако, как по мне, разница довольно умозрительная. Нужно
проводить a/b тесты и выяснять, что лучше. Если у нас онлайн-сервис, было бы полезно делать ux аналитику(ещё до ab), 
вдруг рекомендации по-просту находятся не в очень подходящем месте. Если оффлайн-магазин, было бы полезно проанализировать
расстановку товаров на полках, как разбиваются товары на категории(также до ab). Также для обоих случаев было бы неплохо иметь
понимание конверсии каналов коммуникации и аналитику по продажам в разрезе этих каналов. 

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


Попробуем улучшить бейзлайны, считая на топ-5000 товаров.

In [31]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [33]:
items = data_train.item_id.unique()
sales = data_train.groupby('item_id')['sales_value'].sum().reset_index().sort_values(by='sales_value', ascending=False)[:5000]

In [36]:
weights = sales['sales_value'].apply(lambda x: np.log(x)).values
weights = np.array([x if x >= 0 else 0 for x in weights])
total = weights.sum()
weights = weights / total

In [37]:
result['weighted_random_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(top_5000, weights, n=5))
result.apply(lambda x: precision_at_k(x['weighted_random_5000'], x['actual'],  5), axis=1).mean()

0.001371204701273262

Ну такое себе. Увеличилось на 10^-3.

Попробуем улучшить item-item recommenders. Формально чтобы улучшить метрику надо брать меньшее количество соседей и неплохо
было бы данные отмасштабировать.

In [38]:
# Заведем фиктивный item_id (если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [99]:
%%time

model = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 997 ms


Cosine recommender.

In [95]:
%%time

model = CosineRecommender(K=3, num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 992 ms


TF IDF Recommender.

Если 2 юзера оба купили очень популярный товар, то это еще не значит,что они похожи   
Если 2 юзера оба купили редкий товар, то они похожи

Занижаем вес популярных товаров при расчете расстояний между пользователями

In [78]:
%%time

model = TFIDFRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)


Wall time: 980 ms


Оценим метрики.

In [100]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()

Wall time: 182 ms


0.02579170747633046

In [96]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])
result.apply(lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1).mean()

Wall time: 189 ms


0.014495592556317303

In [79]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])
result.apply(lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1).mean()

Wall time: 203 ms


0.014985308521057754

По сравнению с K=5 удалось улучшить метрики. Если выбирать K>5, то метрики будут хуже. Это связано с тем, что при меньшем k 
алгоритмы больше стремятся вывести в рекомендации историю пользователя, что отчётливо видно в item-item recommender(результат увеличился в два раза), 
где нет никаких дополнительных ограничений.

### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.